[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/neuralangelo-colab/blob/main/colmap_colab.ipynb)

In [ ]:
!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/colmap.zip
!unzip /content/colmap.zip -d colmap

!cp -r /content/colmap/lib/. /usr/local/lib
!chmod 755 /content/colmap/bin/colmap
!cp -r /content/colmap/bin/. /usr/local/bin

!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/fox.zip -O /content/fox.zip
!unzip /content/fox.zip -d /content

!pip install commentjson
%cd /content

In [ ]:
!mkdir /content/fox/sparse /content/fox/dense

!colmap feature_extractor \
    --database_path /content/fox/database.db \
    --image_path /content/fox \
    --ImageReader.camera_model=RADIAL \
    --SiftExtraction.use_gpu=true \
    --SiftExtraction.num_threads=32 \
    --ImageReader.single_camera=true # assuming single camera

!colmap sequential_matcher \
    --database_path /content/fox/database.db \
    --SiftMatching.use_gpu=true

!colmap mapper \
    --database_path /content/fox/database.db \
    --image_path /content/fox \
    --output_path /content/fox/sparse

!colmap image_undistorter \
    --image_path /content/fox \
    --input_path /content/fox/sparse/0 \
    --output_path /content/fox/dense \
    --output_type COLMAP \
    --max_image_size 2000

In [ ]:
%cd /content
!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/instant-ngp-v2.zip
!unzip /content/instant-ngp-v2.zip -d instant-ngp

!pip install commentjson huggingface-hub
%cd /content/instant-ngp

import os
scene_path = "/content/fox" #@param {type:"string"}
train_path = "/content/train"
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)
!rm -rf {train_path}
!mkdir {train_path}
!cp -r {scene_path}/. /content/train
transforms_path = os.path.join(train_path, f"transforms.json")
!python ./scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 4 --images {train_path} --out {transforms_path}